# Assignment Customer Satisfaction and Sentiment Analysis


## Objective

You are a data analyst of a consulting company that provides customer insight regarding multiple ticketing system, such as JIRA and Zoho Desk. Your team gather surveys to customers regarding their ticketing system's performance. Your role in the team is to gather reports regarding customer satisfaction and sentiment analysis into a single dashboard and present your insight.

Analyze the following metrics and other insight you can find in the dataset:

- Survey response rate
- Customer Satisfaction score (CSAT)
- Customer Effort Score (CES)
- Net Promoter Score (NPS)
- Sentiment Analysis



## Data Preparation

In [ ]:
import numpy as np
import pandas as pd
import os

pd.options.display.max_columns = 999
pd.options.display.float_format = "{:.2f}".format

### Access to Local

Write where you put the data in your local.

In [ ]:
data=pd.read_csv('/content/DisneylandReviews.csv',encoding='latin1')

### Read Data

Read the file **DisneylandReviews.csv**

In [ ]:
# Read Data
data.head()

,Review_ID,Rating,Year_Month,Reviewer_Location,Review_Text,Branch
0,670772142,4,2019-4,Australia,If you've ever been to Disneyland anywhere you...,Disneyland_HongKong
1,670682799,4,2019-5,Philippines,Its been a while since d last time we visit HK...,Disneyland_HongKong
2,670623270,4,2019-4,United Arab Emirates,Thanks God it wasn t too hot or too humid wh...,Disneyland_HongKong
3,670607911,4,2019-4,Australia,HK Disneyland is a great compact park. Unfortu...,Disneyland_HongKong
4,670607296,4,2019-4,United Kingdom,"the location is not in the city, took around 1...",Disneyland_HongKong


In [ ]:
# How many customer responded to the survey?
response_count = data['Rating'].notna().sum()

# Output the result
print(f"Jumlah pelanggan yang mengisi survei: {response_count}")

Jumlah pelanggan yang mengisi survei: 42656


The following is the dictionary for the data, survey is only valid if all of the survey questions and text review is not blank (null):

**General Information**
- Review_ID: unique id given to each review
- Rating: ranging from 1 (unsatisfied) to 5 (satisfied)
- Year_Month: when the reviewer visited the theme park
- Reviewer_Location: country of origin of visitor
- Review_Text: comments made by visitor
- Disneyland_Branch: location of Disneyland Park


In [ ]:
# Check the type of data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42656 entries, 0 to 42655
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Review_ID          42656 non-null  int64 
 1   Rating             42656 non-null  int64 
 2   Year_Month         42656 non-null  object
 3   Reviewer_Location  42656 non-null  object
 4   Review_Text        42656 non-null  object
 5   Branch             42656 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.0+ MB


### Data Cleansing

Check column Year Month

Delete missing value in Year_month column

In [ ]:
# Check unique values in Year_Month to identify problematic entries
print(data['Year_Month'].unique())

['2019-4' '2019-5' '2019-3' '2018-9' '2019-1' '2018-11' '2018-12' '2018-6'
 '2019-2' '2018-10' '2018-8' '2018-5' '2018-4' '2018-7' '2018-3' '2018-2'
 'missing' '2018-1' '2017-12' '2017-11' '2017-6' '2017-10' '2017-9'
 '2017-8' '2017-7' '2017-5' '2017-4' '2017-3' '2017-2' '2017-1' '2016-12'
 '2016-11' '2016-10' '2016-9' '2016-8' '2016-7' '2016-6' '2016-5' '2016-4'
 '2016-3' '2016-2' '2016-1' '2015-12' '2015-11' '2015-10' '2015-9'
 '2015-8' '2015-7' '2015-6' '2015-5' '2015-4' '2015-3' '2015-2' '2015-1'
 '2014-12' '2014-11' '2014-10' '2014-9' '2014-8' '2014-7' '2014-6'
 '2014-5' '2014-4' '2014-3' '2014-2' '2014-1' '2013-12' '2013-11'
 '2013-10' '2013-9' '2013-8' '2013-7' '2013-6' '2013-5' '2013-4' '2013-3'
 '2013-2' '2013-1' '2012-12' '2012-11' '2012-10' '2012-9' '2012-8'
 '2012-7' '2012-6' '2012-5' '2012-4' '2012-3' '2012-2' '2012-1' '2011-12'
 '2011-11' '2011-10' '2011-9' '2011-8' '2011-7' '2011-6' '2011-5' '2011-4'
 '2011-3' '2011-2' '2011-1' '2010-12' '2010-11' '2010-10' '2010-9'
 '20

In [ ]:
# Change value 'missing' with NaT (Not a Time)
data['Year_Month'] = data['Year_Month'].replace('missing', pd.NaT)

In [ ]:
# Adding a zero in front of months that only have one digit,'2019-1' become '2019-01'
data['Year_Month'] = data['Year_Month'].apply(lambda x: f'{x[:4]}-{x[5:].zfill(2)}' if isinstance(x, str) else x)


Convert the date column into proper date_time format.

In [ ]:
data.head(100)

,Review_ID,Rating,Year_Month,Reviewer_Location,Review_Text,Branch
0,670772142,4,2019-04,Australia,If you've ever been to Disneyland anywhere you...,Disneyland_HongKong
1,670682799,4,2019-05,Philippines,Its been a while since d last time we visit HK...,Disneyland_HongKong
2,670623270,4,2019-04,United Arab Emirates,Thanks God it wasn t too hot or too humid wh...,Disneyland_HongKong
3,670607911,4,2019-04,Australia,HK Disneyland is a great compact park. Unfortu...,Disneyland_HongKong
4,670607296,4,2019-04,United Kingdom,"the location is not in the city, took around 1...",Disneyland_HongKong
...,...,...,...,...,...,...
95,663350807,5,2019-03,Indonesia,Fantastic Nice place and kids friendly Easly...,Disneyland_HongKong
96,663079041,5,2019-03,Australia,What is there not to love about Disneyland? Th...,Disneyland_HongKong
97,662971560,4,2019-03,Singapore,Friendly staff and cooling weather! Used Klook...,Disneyland_HongKong
98,662786512,4,2019-03,Malaysia,Smallest Disneyland and spent a day here. Havi...,Disneyland_HongKong


In [ ]:

# Menghapus baris dengan nilai kosong pada kolom 'Year_Month'
data = data.dropna(subset=['Year_Month'])

In [ ]:
print(data.dtypes)


Review_ID             int64
Rating                int64
Year_Month           object
Reviewer_Location    object
Review_Text          object
Branch               object
dtype: object


Check Column Branch

Delete missing value in Branch column

In [ ]:
# Check unique values in Branch to identify problematic entries
print(data['Branch'].unique())

['Disneyland_HongKong' 'Disneyland_California' 'Disneyland_Paris']


In [ ]:
# Ganti string kosong dengan NaN terlebih dahulu
data['Branch'].replace('', pd.NA, inplace=True)

# Lalu hapus baris yang kosong
df_cleaned = data.dropna(subset=['Branch'])


<ipython-input-199-86301d343b45>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Branch'].replace('', pd.NA, inplace=True)


In [ ]:
# Filter review yang mengandung karakter '*' atau '**'
filtered_df = data[data['Review_Text'].str.contains(r'\*', regex=True)]
# Menghapus semua * dan ** dari Review_Text
data['Review_Text'] = data['Review_Text'].str.replace(r'\*+', '', regex=True)

In [ ]:
# Simpan ke file Excel
data.to_excel('Final_DisneylandReviews.xlsx', index=False)

## Survey Analysis

### Response Rate

Start by analyzing how many customers has filled the survey, indicated by whether the overall_rating is not blank.

In [ ]:
# How many customer responded to the survey?
response_count = data['Rating'].notna().sum()

# Output the result
print(f"Jumlah pelanggan yang mengisi survei: {response_count}")

Jumlah pelanggan yang mengisi survei: 40043


Create a new dataframe that consists only of those who have responded the survey to simplify calculating the CSAT, CES, and NPS Score.

In [ ]:
# Responded Customer
responded_customers_df = data[data['Rating'].notna()]

# Output the first few rows of the new dataframe to verify
print(responded_customers_df.head())

   Review_ID  Rating Year_Month     Reviewer_Location  \
0  670772142       4 2019-04-01             Australia   
1  670682799       4 2019-05-01           Philippines   
2  670623270       4 2019-04-01  United Arab Emirates   
3  670607911       4 2019-04-01             Australia   
4  670607296       4 2019-04-01        United Kingdom   

                                         Review_Text               Branch  
0  If you've ever been to Disneyland anywhere you...  Disneyland_HongKong  
1  Its been a while since d last time we visit HK...  Disneyland_HongKong  
2  Thanks God it wasn   t too hot or too humid wh...  Disneyland_HongKong  
3  HK Disneyland is a great compact park. Unfortu...  Disneyland_HongKong  
4  the location is not in the city, took around 1...  Disneyland_HongKong  


### CSAT Score

Measure the customer's overall satisfaction score (CSAT) with the following formula:

$$
CSAT = \frac{\Sigma\ total\ satisfaction\ score}{number\ of\ responded\ customer \times \max\ rating}
$$

The max rating is inserted to convert the CSAT score into percentage.

CSAT score can be classified into categories based on the result. There is no absolute threshold for each categories but the following is the common threshold:

- \>= 90%: Excellent
- 75%-90%: Good
- 60-75%: Fair
- \<60%: Poor

In [ ]:
# CSAT Score
# Hitung jumlah masing-masing rating per cabang
rating_distribution = data.groupby(['Branch', 'Rating']).size().unstack(fill_value=0)

# Tambahkan total review per cabang
rating_distribution['Total_Reviews'] = rating_distribution.sum(axis=1)

# Tambahkan CSAT Score: rating 4 dan 5
rating_distribution['CSAT_Percentage'] = ((rating_distribution[4] + rating_distribution[5]) /
                                           rating_distribution['Total_Reviews']) * 100

# Tampilkan hasil
print(rating_distribution)

Rating                   1    2     3     4      5  Total_Reviews  \
Branch                                                              
Disneyland_California  450  685  1552  3713  11802          18202   
Disneyland_HongKong    152  307  1297  3053   4338           9147   
Disneyland_Paris       736  937  1933  3320   5768          12694   

Rating                 CSAT_Percentage  
Branch                                  
Disneyland_California            85.24  
Disneyland_HongKong              80.80  
Disneyland_Paris                 71.59  


Measure the satisfaction score for the following attributes:

- customer service
- features
- value for money

In [ ]:
# Satisfaction Score for Attributes


### CES Score

Measure CES with the following formula


$$
CES = \frac{\Sigma\ total\ effort\ score}{number\ of\ responded\ customer \times \max\ rating}
$$

In [ ]:
# CES Score
# Tentukan rating maksimum (misalnya rating berkisar antara 1 hingga 5)
max_rating = 5

# Fungsi untuk menghitung CES per cabang
def calculate_ces_for_branch(df):
    total_effort_score = df['Rating'].sum()  # Jumlah rating untuk cabang ini
    number_of_responded_customers = df['Rating'].count()  # Jumlah pelanggan yang merespon
    ces_score = (total_effort_score / (number_of_responded_customers * max_rating)) * 100  # Hitung CES
    return ces_score

# Kelompokkan berdasarkan cabang dan hitung CES untuk setiap cabang
branch_ces_scores = responded_customers_df.groupby('Branch').apply(calculate_ces_for_branch)

# Fungsi untuk mengklasifikasikan CES
def classify_ces(ces_score):
    if ces_score >= 80:
        return "Excellent"
    elif ces_score >= 60:
        return "Good"
    else:
        return "Poor"

# Terapkan klasifikasi CES ke setiap cabang
branch_ces_categories = branch_ces_scores.apply(classify_ces)

# Gabungkan hasil CES dengan kategori ke dalam DataFrame
ces_comparison = pd.DataFrame({
    'CES Score': branch_ces_scores,
    'CES Category': branch_ces_categories
})

# Tampilkan perbandingan CES antar cabang
print("CES Scores and Categories by Disneyland Branch:")
print(ces_comparison)


CES Scores and Categories by Disneyland Branch:
                       CES Score CES Category
Branch                                       
Disneyland_California      88.11    Excellent
Disneyland_HongKong        84.08    Excellent
Disneyland_Paris           79.20         Good


<ipython-input-27-a6612341600f>:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  branch_ces_scores = responded_customers_df.groupby('Branch').apply(calculate_ces_for_branch)


In [ ]:
# List kata kunci
customer_service_keywords = ['staff', 'help', 'friendly', 'service', 'professional']
features_keywords = ['rides', 'attractions', 'parades', 'fireworks', 'character']
value_keywords = ['expensive', 'cheap', 'worth', 'money']

# Ubah teks ke lowercase
data['Review_Text'] = data['Review_Text'].astype(str).str.lower()

# Buat fungsi bantu
def contains_keyword(text, keywords):
    return any(keyword in text for keyword in keywords)

# Tambahkan kolom indikator untuk masing-masing kategori
data['Customer_Service'] = data['Review_Text'].apply(lambda x: contains_keyword(x, customer_service_keywords))
data['Features'] = data['Review_Text'].apply(lambda x: contains_keyword(x, features_keywords))
data['Value'] = data['Review_Text'].apply(lambda x: contains_keyword(x, value_keywords))

# Hitung persentase per branch
result = data.groupby('Branch').agg(
    Total_Reviews=('Review_Text', 'count'),
    Customer_Service_Count=('Customer_Service', 'sum'),
    Features_Count=('Features', 'sum'),
    Value_Count=('Value', 'sum')
)

# Hitung persentase
result['Customer_Service (%)'] = (result['Customer_Service_Count'] / result['Total_Reviews']) * 100
result['Features (%)'] = (result['Features_Count'] / result['Total_Reviews']) * 100
result['Value for Money (%)'] = (result['Value_Count'] / result['Total_Reviews']) * 100

# Pilih hanya kolom persen
final_result = result[['Customer_Service (%)', 'Features (%)', 'Value for Money (%)']].round(2)

# Tampilkan hasil
print(final_result)

                       Customer_Service (%)  Features (%)  Value for Money (%)
Branch                                                                        
Disneyland_California                 21.78         57.02                24.40
Disneyland_HongKong                   17.33         65.35                26.90
Disneyland_Paris                      33.57         73.14                39.82


### NPS Score

To calculate the NPS score, first we must convert the **would_you_recommend** column into proper NPS Category based on the rating value:

* Promoter: Rating 9-10
* Passive: Rating 7-8
* Detractor: Rating < 7

In [ ]:
# Category NPS



Calculate the NPS Score with the following formula

$$
NPS = \frac{Promoter - Detractor}{Total\ Survey\ Responded}
$$

In [ ]:
# NPS Score
# Tentukan rating untuk kategori Promoters, Passives, dan Detractors
def calculate_nps_category(Rating):
    if Rating >= 4:  # Promoters
        return 'Promoter'
    elif Rating == 3:  # Passives
        return 'Passive'
    else:  # Detractors
        return 'Detractor'

# Terapkan fungsi untuk menghitung kategori NPS ke kolom baru 'NPS_Category'
responded_customers_df['NPS_Category'] = responded_customers_df['Rating'].apply(calculate_nps_category)

# Hitung jumlah Promoters, Passives, dan Detractors
nps_counts = responded_customers_df['NPS_Category'].value_counts()

# Hitung persentase Promoters dan Detractors
total_responses = len(responded_customers_df)
promoters_percentage = (nps_counts.get('Promoter', 0) / total_responses) * 100
detractors_percentage = (nps_counts.get('Detractor', 0) / total_responses) * 100

# Hitung NPS
nps_score = promoters_percentage - detractors_percentage

# Fungsi untuk menghitung NPS per cabang
def calculate_nps_for_branch(df):
    nps_counts = df['NPS_Category'].value_counts()
    total_responses = len(df)
    promoters_percentage = (nps_counts.get('Promoter', 0) / total_responses) * 100
    detractors_percentage = (nps_counts.get('Detractor', 0) / total_responses) * 100
    nps_score = promoters_percentage - detractors_percentage
    return nps_score

# Kelompokkan berdasarkan cabang dan hitung NPS untuk setiap cabang
branch_nps_scores = responded_customers_df.groupby('Branch').apply(calculate_nps_for_branch)

# Tampilkan NPS per cabang
print("NPS Scores by Disneyland Branch:")
print(branch_nps_scores)

NPS Scores by Disneyland Branch:
Branch
Disneyland_California   78.60
Disneyland_HongKong     75.25
Disneyland_Paris        57.25
dtype: float64


<ipython-input-33-5ee639411243>:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  branch_nps_scores = responded_customers_df.groupby('Branch').apply(calculate_nps_for_branch)


NPS score can be ranging from -100 (when all customers are detractor) to 100 (when all customers are promoter).

NPS Score can be classified into categories based on the following threshold:

- \>= 70: Excellent
- 50-69: Very Good
- 30-49: Good
- 0-29: Average
- \< 0: Poor

## Sentiment Analysis

Create a new dataframe with no blank overall_text.

In [ ]:
# Create new dataframe
# Asumsi bahwa 'responded_customers_df' adalah DataFrame yang berisi data respon pelanggan

# Membuat DataFrame baru yang hanya berisi baris dimana 'Review_Text' tidak kosong (bukan null)
non_blank_reviews_df = responded_customers_df[responded_customers_df['Review_Text'].notnull()]

# Jika Anda ingin menghapus baris yang memiliki teks kosong ('') atau hanya spasi
non_blank_reviews_df = non_blank_reviews_df[non_blank_reviews_df['Review_Text'].str.strip() != '']

# Tampilkan DataFrame baru yang hanya berisi review yang tidak kosong
print(non_blank_reviews_df)

       Review_ID  Rating Year_Month     Reviewer_Location  \
0      670772142       4 2019-04-01             Australia   
1      670682799       4 2019-05-01           Philippines   
2      670623270       4 2019-04-01  United Arab Emirates   
3      670607911       4 2019-04-01             Australia   
4      670607296       4 2019-04-01        United Kingdom   
...          ...     ...        ...                   ...   
42651    1765031       5        NaT        United Kingdom   
42652    1659553       5        NaT                Canada   
42653    1645894       5        NaT          South Africa   
42654    1618637       4        NaT         United States   
42655    1536786       4        NaT        United Kingdom   

                                             Review_Text               Branch  \
0      If you've ever been to Disneyland anywhere you...  Disneyland_HongKong   
1      Its been a while since d last time we visit HK...  Disneyland_HongKong   
2      Thanks God it was

### Text Cleansing

In order to get more accurate sentiment, several text cleansing need to be done. However, in most of recent sentiment analysis models and algorithm, the only text cleansing needed are as follows:

* Clean double whitespace
* Clean URL/website
* Clean username (mostly in social media or digital text)

In [ ]:
import re

def cleansing_text(x):
  # clean double whitespace
  out_text = ' '.join(x.split())

  # clean url
  out_text = re.sub(r"http\S+|www\S+|https\S+", 'http', out_text)

  # clean username
  out_text = re.sub(r"@\S+", '@user', out_text)

  return(out_text)

cleansing_text(" Doesn't  dissapoint. The car       was great. It was the best car rental experiences I've had! Salute to @jone who recommend https:/rental.com")

In [ ]:
# apply cleansing to review


                                         Review_Text  \
0  If you've ever been to Disneyland anywhere you...   
1  Its been a while since d last time we visit HK...   
2  Thanks God it wasn   t too hot or too humid wh...   
3  HK Disneyland is a great compact park. Unfortu...   
4  the location is not in the city, took around 1...   

                                 Cleaned_Review_Text  
0  If you've ever been to Disneyland anywhere you...  
1  Its been a while since d last time we visit HK...  
2  Thanks God it wasn t too hot or too humid when...  
3  HK Disneyland is a great compact park. Unfortu...  
4  the location is not in the city, took around 1...  


### Sentiment Analysis

Create a sentiment categories using algorithm of your own choice.

In [ ]:
# Sentiment Algorithm


Check the number of data by sentiment.

In [ ]:
# Number of Sentiment


Sentimen Analysis using 'Review_Text' group by Branch

In [ ]:
from textblob import TextBlob
import pandas as pd

# Assuming 'responded_customers_df' is the DataFrame of customers who have responded to the survey

# Define keyword lists for each attribute
customer_service_keywords = ['staff', 'help', 'friendly', 'service', 'assistance']
features_keywords = ['rides', 'attractions']
value_for_money_keywords = ['expensive', 'cheap', 'worth']

# Function to calculate sentiment polarity
def get_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity

# Function to check if a review contains relevant keywords and analyze sentiment
def analyze_sentiment_based_on_keywords(text, keywords):
    # Convert text to lowercase for case-insensitive matching
    text = text.lower()

    # Check if any of the keywords are in the review text
    if any(keyword in text for keyword in keywords):
        sentiment_score = get_sentiment(text)
        sentiment = 'Positive' if sentiment_score > 0 else ('Negative' if sentiment_score < 0 else 'Neutral')
        return sentiment, sentiment_score
    else:
        return 'Not Relevant', 0

# Apply the function to the review text for each attribute
responded_customers_df['Customer_Service_Sentiment'], responded_customers_df['Customer_Service_Sentiment_Score'] = zip(
    *data['Review_Text'].apply(lambda x: analyze_sentiment_based_on_keywords(x, customer_service_keywords))
)

responded_customers_df['Features_Sentiment'], responded_customers_df['Features_Sentiment_Score'] = zip(
    *responded_customers_df['Review_Text'].apply(lambda x: analyze_sentiment_based_on_keywords(x, features_keywords))
)

responded_customers_df['Value_for_Money_Sentiment'], responded_customers_df['Value_for_Money_Sentiment_Score'] = zip(
    *responded_customers_df['Review_Text'].apply(lambda x: analyze_sentiment_based_on_keywords(x, value_for_money_keywords))
)

# Group by Disneyland Branch and calculate sentiment counts for each branch
branch_sentiment_counts = responded_customers_df.groupby('Branch').agg(
    Customer_Service_Positive=('Customer_Service_Sentiment', lambda x: (x == 'Positive').sum()),
    Customer_Service_Negative=('Customer_Service_Sentiment', lambda x: (x == 'Negative').sum()),
    Customer_Service_Neutral=('Customer_Service_Sentiment', lambda x: (x == 'Neutral').sum()),

    Features_Positive=('Features_Sentiment', lambda x: (x == 'Positive').sum()),
    Features_Negative=('Features_Sentiment', lambda x: (x == 'Negative').sum()),
    Features_Neutral=('Features_Sentiment', lambda x: (x == 'Neutral').sum()),

    Value_for_Money_Positive=('Value_for_Money_Sentiment', lambda x: (x == 'Positive').sum()),
    Value_for_Money_Negative=('Value_for_Money_Sentiment', lambda x: (x == 'Negative').sum()),
    Value_for_Money_Neutral=('Value_for_Money_Sentiment', lambda x: (x == 'Neutral').sum())
)

# Output the sentiment counts by Disneyland branch
print("Sentiment counts by Disneyland Branch:")
print(branch_sentiment_counts)


Sentiment counts by Disneyland Branch:
                       Customer_Service_Positive  Customer_Service_Negative  \
Branch                                                                        
Disneyland_California                       3965                        242   
Disneyland_HongKong                         1566                        102   
Disneyland_Paris                            4096                        584   

                       Customer_Service_Neutral  Features_Positive  \
Branch                                                               
Disneyland_California                         7               8842   
Disneyland_HongKong                           2               4815   
Disneyland_Paris                              4               7706   

                       Features_Negative  Features_Neutral  \
Branch                                                       
Disneyland_California                659                27   
Disneyland_HongKong         

## Finalize Data for Reporting

Save the review data with NPS category and sentiment information to new csv for the dashboard.

In [ ]:
# Save Data
